### Theory of SVD

Singular Value Decomposition (SVD) is a fundamental matrix factorization technique in linear algebra. It decomposes any  m x n matrix  A  into three specific matrices:

A = U Σ V^T 

where:

1. U: An  m x m  orthogonal matrix.
   - The columns of  U  are the left singular vectors of  A .
   - These vectors form an orthonormal basis for the column space of  A .

2.  Σ : An  m x n  diagonal matrix (or a matrix with zeros beyond the diagonal).
   - The diagonal entries of  Σ  are the singular values of  A , which are non-negative and typically arranged in descending order.
   - Singular values represent the "weight" of each corresponding singular vector.

3.  V^T : An  n x n  orthogonal matrix (V Transpose).
   - The rows of  V^T  are the right singular vectors of  A .
   - These vectors form an orthonormal basis for the row space of  A .
   - The matrices  U  and  V  are orthogonal, meaning  U^T U = I  and  V^T V = I , where  I  is the identity matrix. This ensures that  U  and  V  are composed of orthonormal vectors.


### Applications:

SVD is widely used in signal processing, data compression (e.g., image compression), noise reduction, and dimensionality reduction techniques such as Principal Component Analysis (PCA). It provides a way to analyze the structure of matrices and solve various problems in numerical linear algebra and statistics.


### SVD Implementation from Scratch
The below code involves calculating the V, U and Sigma matrices as per the usual method of calculating them. To calculate the eigenvectors and eigenvalues, an in-built function was used but the rest of the process is procedural. The matrices are then multiplied to verify if the original matrix A is given or not.

The computation time is also calculated.

In [25]:
import numpy as np
import time

def SVD(A):
    #V
    ATransA = np.dot(A.T, A)

    eig_V, V = np.linalg.eig(ATransA)  
    eig_V_sorted_indices = np.argsort(eig_V)[::-1]

    V1 = V[:, eig_V_sorted_indices]
    eig_V1 = eig_V[eig_V_sorted_indices]

    #U
    AATrans = np.dot(A, A.T)

    eig_U, U = np.linalg.eig(AATrans)
    eig_U_sorted_indices = np.argsort(eig_U)[::-1]

    U1 = U[:, eig_U_sorted_indices]
    eig_U1 = eig_U[eig_U_sorted_indices]

    #Sigma
    Sigma = np.zeros_like(A, dtype=float)
    for i in np.arange(min(A.shape)):
        Sigma[i, i] = np.sqrt(eig_V1[i])    
    
    return U1, Sigma, V1.T

A = np.array([[3, 2, 1], [2, 1, 5]])
# A = np.array([[4, 2, 0], [1, 5, 6]])

# print(A)
start_time = time.perf_counter()
U, Sigma, VT = SVD(A)
end_time = time.perf_counter()

print("U Matrix")
print(U)
print("Sigma")
print(Sigma)
print("V^T")
print(VT)

u_sigma = np.dot(U, Sigma)
print(u_sigma)

A_reconstructed = U @ Sigma @ VT
print("A_reconstructed")
print(A_reconstructed)

print(f"Time Taken = {(end_time - start_time)*1e3} ms")

U Matrix
[[-0.48780251 -0.87295402]
 [-0.87295402  0.48780251]]
Sigma
[[6.10445227 0.         0.        ]
 [0.         2.59531549 0.        ]]
V^T
[[-0.52573358 -0.30282144 -0.7949235 ]
 [-0.63316273 -0.48476015  0.6034174 ]
 [ 0.56807496 -0.82055272 -0.06311944]]
[[-2.97776713 -2.26559108  0.        ]
 [-5.32890612  1.2660014   0.        ]]
A_reconstructed
[[3. 2. 1.]
 [2. 1. 5.]]
Time Taken = 0.649199999315897 ms


### Comparing with in-built function
The above code output is now compared with the SVD function provided in NumPy's Linear Algebra library. The outputs match.

The computation time is considerable less with the inbuilt function than the manual implementation.

In [22]:
A = np.array([[3, 2, 1], [2, 1, 4]])

start_time = time.perf_counter()
U, Sigma, VT = np.linalg.svd(A)
end_time = time.perf_counter()

print(U, Sigma, VT)
print(f"Time Taken = {(end_time - start_time)*1e3} ms")

[[-0.6 -0.8]
 [-0.8  0.6]] [5.47722558 2.23606798] [[-0.62075223 -0.36514837 -0.69378191]
 [-0.53665631 -0.4472136   0.71554175]
 [-0.57154761  0.81649658  0.08164966]]
Time Taken = 0.2015999998548068 ms


### Image Compression and Quality Enhancement

The below code uses the above SVD function to compress images. Various values of k are chosen to compare the compression and quality. The top k singular values of U, Sigma and V^T are taken and the image matrix is constructed using these. Then it is saved as an image.

In [36]:
from PIL import Image

def compress_image(path, k):
    image = Image.open(path).convert('L')
    A = np.array(image, dtype=float)
    # print(A.shape)

    U, Sigma, Vt = SVD(A)
    
    U_k = U[:, :k]
    Sigma_k = Sigma[:k, :k]
    Vt_k = Vt[:k, :]

    A_k = U_k @ Sigma_k @ Vt_k

    A_k = np.clip(A_k, 0, 255).astype(np.uint8)

    # compressed_image = Image.fromarray(A_k)
    # compressed_image.save(f"compressed_k_{k}.jpg")
    # compressed_image.show()
    compressed_image_name = f"compressed1_k_{k}.jpg"
    compressed_image = Image.fromarray(A_k)
    compressed_image.save(compressed_image_name, "JPEG", quality = 20)

# compress_image('quality.jpeg', 50)
compress_image('quality1.jpeg', 1000)

(1500, 1000)


C:\Users\Aroshish\AppData\Local\Temp\ipykernel_18588\1107230438.py:16: ComplexWarning: Casting complex values to real discards the imaginary part
  A_k = np.clip(A_k, 0, 255).astype(np.uint8)


### Report on Findings

##### Overview
Singular Value Decomposition (SVD) is a powerful mathematical tool for dimensionality reduction and data compression. In this report, we investigate the effects of compressing images using SVD by retaining only a subset of the singular values and corresponding vectors. We explore different values of k (number of singular values kept), and observe the trade-offs between image quality and file size.

##### Methodology
1. SVD Application: For each image, we decompose it using SVD into matrices U, Sigma and V^T. By keeping only the top k singular values and their corresponding vectors, we reconstructed a compressed version of the image.
2. Image Reconstruction: We reconstruct the image using the reduced matrices and save the results as JPG.
3. Comparison: We compare the reconstructed images with the original image for various values of k. We use visual inspection and file size metrics to assess the quality of the compressed images.

Values of k: 50, 100, 200, 500, 1000

##### Observations
1. Quality vs k Value:
* Low k Values (e.g. 50): The compressed images show significant loss of detail. Important features and fine textures are often blurred or lost, resulting in loss of image quality.
* Moderate k Values (e.g. 100, 200): As k increases, more details are preserved. The images appear clearer with fewer artifacts, but some loss of fine detail are still present.
* High k Values (e.g., 500, 1000): The images closely resemble the original with minimal perceptible loss of quality. The reconstruction retains most of the details and textures, showing a clear trade-off between compression and fidelity.

2. File Size:

* Low k Values: Smaller file sizes due to higher compression, but accompanied by significant quality loss.
* Higher k Values: Larger file sizes but improved image quality. The size increase is proportional to the number of singular values retained.


##### Trade-Offs
Compression Ratio: Lower k values provide higher compression ratios but at the cost of image quality. Higher k values reduce compression but retain more detail.

Quality: The visual quality of the image improves as k increases, with a more accurate representation of the original image. However, the file size also increases with k.

##### Conclusion
The choice of k in SVD-based image compression involves a trade-off between compression ratio and image quality. Lower k values offer greater compression but reduce image clarity. Higher k values retain more detail, resulting in higher quality images but with larger file sizes. For practical applications, the choice of k should balance the need for compression with the acceptable level of image quality loss.